# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
#import Pkg; 
#Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [3]:
function TestsSondabilite_LP(BS, BI, faisable, model2, xSup, xInf, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(!faisable)#Test de faisabilite
        TA=true
        #println("TA")
    elseif(BS <= BestProfit) #Test d'optimalite
        TO=true
        #println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in xSup]-xSup) .<= fill(10^-5, size(xSup))) && all(i->(i>=0&&i<=1),xSup)
        ) #Test de resolution #vérifie que les solutions sont entières et compris entre 0 et 1(cette deuxième condition n'es utile que lorsque c'est BS1 qui est utilisé)
        TR=true
        #println("TR")
        #if (value(benef) >= BestProfit)
        if (BS >= BestProfit)
            Bestsol = copy(xSup)
            #BestProfit=value(benef)
            BestProfit=BS
            #println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
    else
        #println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [4]:

function SeparerNoeud_lexicographic_depthfirst!(obj,listobjs, listvals, n)
    
    #println("\nbranch on object ", obj, "\n") #on se branche sur l'objet obj qui est celui avec le meilleure ratio déterminé dans le calcul des BS

    # maxBS exploration strategy: on choisit le noeud qui a le meilleure BS qui est celui où on ajoute l'objet où on se branche
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop 
end

ExplorerAutreNoeud_depthfirst! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [ ]:
function MajModele_LP!(x, listobjs, listvals)
    for i in 1:length(listobjs)
        x[listobjs[i]]= listvals[i]
    end
end

function Reset_allLP!(x)
    for i in 1:length(x)
        x[i]=0.0
    end
end

function CreationModele_LP(price, weight, capacity)
# ROOT NODE
    
    n=length(price)
    max(x) = sum(price[i]*x[i] for i in 1:n)
    cond(x) = sum(weight[i]*x[i] for i in 1:n) <= capacity
    x = zeros(n,1)
    model2 = Dict("x"=>x,"objectif"=>max, "contrainte" => cond) 

    return model2, x
end

function BS1(xSup,xInf,capacity,price,weight,listobjs,listvals)
    BS = 0
    BI = 0
    n = length(xSup)
    capacityTmp = capacity #capacity restante du sac à chaque instant 
    selectionnable = [] #objet qui n'ont pas encore été décidé si pris ou non
    ratio = Dict() #dictionnaire associant à chaque ratio son objet correspondant
    idObj = 1
    for i in listobjs
        capacityTmp = capacityTmp - listvals[idObj]*weight[i]
        idObj = idObj+1
    end
    for i in 1:n
        if(!(i in listobjs))
            push!(selectionnable,i)
            ratio[price[i]/weight[i]] = i
        end       
    end
    obj = 1
    if(selectionnable !=[])
        ratiomax = maximum(ratio)[1]
        obj = ratio[ratiomax] #objet sur lequel est effectué le branchement
        xSup[obj] = capacityTmp/weight[obj] #autant de fois possible le meilleur objet
    end
    return xSup,xInf,obj
end

function BS2(xSup,xInf,capacity,price,weight,listobjs,listvals)
    BS = 0
    BI = 0
    n = length(xSup)
    capacityTmp = capacity #capacity restante du sac à chaque instant 
    selectionnable = [] #objet qui n'ont pas encore été décidé si pris ou non
    ratio = Dict() #dictionnaire associant à chaque ratio son objet correspondant
    idObj = 1 #index de l'objet dans la list des valeurs
    
    for i in listobjs
        capacityTmp = capacityTmp - listvals[idObj]*weight[i]
        idObj = idObj+1
    end
    for i in 1:n
        if(!(i in listobjs))
            push!(selectionnable,i)
            ratio[price[i]/weight[i]] = i
        end       
    end
    objChoisi = 1
    if(selectionnable !=[])
        objChoisi = ratio[maximum(ratio)[1]]
        while ratio!=Dict()
            ratiomax = maximum(ratio)[1] #toujours essayé d'ajouter l'objet de meilleure ratio dans les selectionnables
            obj = ratio[ratiomax]
            delete!(ratio,ratiomax) #retiré l'objet et son ratio vu qu'il sera ajouté dans le sac donc n'est plus sélectionnable
            if(capacityTmp-weight[obj] < 0) #si plus de place dans le sac mettre suffisament(<1) le meilleur objet actuel pour le remplir
                xSup[obj] = capacityTmp/weight[obj]
                break
            else #sinon mettre l'objet en entier
                xSup[obj] = 1.0
                xInf[obj] = 1.0
                capacityTmp = capacityTmp-weight[obj]
            end
        end
        
    end
    return xSup,xInf,objChoisi
end

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [6]:

function SolveKnapInstance(filename,Borne)

    price, weight, capacity = readKnaptxtInstance(filename)

    model2, x = CreationModele_LP(price, weight, capacity)
    n = length(x)
    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]

    current_node_number=0
    stop = false
    
    while(!stop)

        #println("\nNode number ", current_node_number, ": \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        
        #create LP of current node
        MajModele_LP!(x, listobjs, listvals)
        
        xInf = copy(x)
        xSup = copy(x)
        #println(model2)
        
        #print("Solve the LP model of the current node to compute its bound: start ... ")
        obj=1
        if Borne == "BS1" #choix de la borne
            xSup,xInf,obj = BS1(xSup,xInf,capacity,price,weight,listobjs,listvals)
        else 
            xSup,xInf,obj = BS2(xSup,xInf,capacity,price,weight,listobjs,listvals)
        end
        
        #println("... end"); 
        BS = model2["objectif"](xSup)
        BI = model2["objectif"](xInf)
        faisable = model2["contrainte"](x) #si la contrainte du model est respecté donc pas plus d'ojet qu'il y'a de place
        #println(": Solution LP")
        
        """if(!faisable)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print("BS :", BS)
            [print("\t ","x[",i,"]","=",xSup[i]) for i in 1:n]
            println("\n")
             print("BI:", BI)
            [print("\t ","x[",i,"]","=",xInf[i]) for i in 1:n]
        end
        println(""); 


        println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")"""

        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_LP(BS, BI, faisable, model2, xSup, xInf, BestProfit, Bestsol)

        is_node_sondable = TA || TO || TR


        if(!is_node_sondable)
            if (BI > BestProfit) #si le noeud est nonsondable mais que sa BI est meilleure que la meilleure solution connue on la met à jour
                Bestsol = copy(xInf)
                #BestProfit=value(benef)
                BestProfit=BI
                #println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
            end
            SeparerNoeud_lexicographic_depthfirst!(obj,listobjs, listvals, length(price))
        else
            stop = ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes)
        end
        
        Reset_allLP!(x)

        current_node_number = current_node_number + 1
    end

    #println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [25]:
InstanceEtu = "instancesETU/KNAPnewformat/profit_ceiling/knapPI_15_50_1000_1_-1005.opb.txt"
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance(InstanceEtu,"BS2")
println("\n******\n\nNombre de Noeud = ",length(trNamenodes),"\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
# graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)


FINISHED

******

Nombre de Noeud = 17361

Optimal value = 1005.0

Optimal x=[1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 1.0; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 1.0; 0.0; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0]


## COMPTE RENDU

### Questions Préliminaires

1. La règle de séparation ici est de choisir les objets par ordre lexicographique dans cette exemple ce sera l'objet 1 puis 2 puis 3 puis 4

2. La méthode est celle de BS2 dans notre cours donc une relaxation linéaire du probléme du sac à dos où x appartient à IR et 0 <= x <= 1
3. TA est validé si la solution est faisable donc ne viole pas les contraintes du modèle

   T0 est validé si la fonction objectif pour la solution actuelle est inférieur au meilleur profit au moment du test
   
   TR est validé si la solution est entière donc chaque valeur d'objet est une valeur entière
   
4. partir dans le noeud où on ajoute l'objet sur lequel on s'est branché avec la règle de séparation

### Code et Analyse

3. Les points clés de mon implémentation ont été pour:
- la règle de Séparation : de donner directement en paramètre de la fonction l'objet sur lequel se brancher (qui est celui avec le meilleur ratio coût/prix) car ce calcul etait effectuer lors du calcul de la BS et BI, pour eviter ainsi de la redondance.
- le TA : de garder les données de x dans le noeud courant pour verifier si la contraintes de notre modèle est respecté.
- le TO : de pouvoir calculer la BorneSup afin de la comparer avec le meilleure profit connue
- le TR : de rajouter une condition sur le faite que les valeurs de x sont compris entre 0 et 1 ce qui est inutile lorsque l'on utilise BS2 mais l'est quand on utilise BS1 qui n'impose pas cette contrainte sur les valeurs des objets (xSup[1] = 2. est possible dans BS1 mais pas dans BS2)
- la statégie d'exploration : de voir que la meilleure BS sera toujours dans le noeud où l'on ajoute l'objet sur lequel on se branche ce qui etait déja le cas dans l'exemple q'on nous a donné donc pas de modification

4. Pour les structures de données on a décidé :
- d'utiliser une dictionnaire pour stocker la contrainte et la fonction objectif de notre modèle de notre modèle.
- d'utiliser 3 vecteurs distincts x,xInf,xSup pour conserver respectivement la valeur des objets dans un noeud, celle des objets permettant de calculer la BS dans ce noeud et celle des objets permettant de calculer la BI dans ce noeud.
- d'utiliser un dictionnaire pour associer les ratios aux objets et permettant plus facilement de récupérer le ratio et l'objet qui lui est associé.

5. On a comparé les performances en utilisant borne 1 et borne 2 sur plusieurs test et on remarque d'abord dans l'ensemble la borne 2 est plus efficace que la borne 1 (qui selon nous est dû au faites du calcul de la borneInf qui est meilleure avec Borne 2) . mais que suivant la façon dont les données sont liés entre elle l'écart sera plus ou moins présent comme par exemple entre des données circle, inverse_strongly_correlated ou encore uncorrelated il y'aura un gros ecart entre la performance des deux algorithmes tandis que quand les données sont similar_weight ou multiple_strongly_corrolated on a quasiment les mêmes performances pour les deux algorithmes.    
Quelques test réalisé: BS1 || BS2  (noeud)  
circle/knapPI_16_20_1000_4_-2893.opb : 15369 || 3627  
inverse_strongly_correlated/knapPI_4_50_1000_5_-1278.opb.txt : 4463 || 893  
uncorrelated/knapPI_1_50_1000_4_-4888.opb.txt : 2665 || 427  
similar_weights\knapPI_9_50_1000_4_-976.opb.txt : 18893 || 18893  
multiple_strongly_correlated\knapPI_14_200_1000_2_-6898.opb.txt : 20833 || 20831

PS : La mise en commentaire dans le code des print et de l'affichage graphique a été mise pour décharger le code d'affichage qui peuvent empêcher qu'il s'affiche en entier s'il y'a trop de noeud et du graphplot qui devient illisible quand il y'a trop de noeud ils pourront être décommenté dans le cas du test.opb pour vérifier les états d'avancement dans le code